In [1]:
import requests, boto3, pandas as pd, os, sys, pprint
from pathlib import Path
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

from entities.RedditAccount import RedditAccount
from entities.DailyUpload import DailyUpload

#TODO: Change path accordingly in handler


# Making the current directory in which this file is in discoverable to python.
# Commenting it here because it will not work in jupyter notebook. It will work in lambda though.
#sys.path.append(os.path.join(os.path.dirname(__file__)))

# Below should be used only in jupyter notebook
sys.path.append('./')


REDDIT_AUTH_URL = 'https://www.reddit.com/api/v1/access_token'
REDDIT_ACCOUNTS_TABLE_NAME = 'RedditAccountsTable-dev'
DAILY_UPLOADS_TABLE = "DailyUploadsTable-dev"
REDDIT_API_URL_TOP = "https://oauth.reddit.com/r/placeholder_value/top"
REDDIT_API_URL_SORT = "https://oauth.reddit.com/r/placeholder_value/sort"

ddb = boto3.client("dynamodb", region_name="ap-south-1")
subreddit='funny'

## Get values from reddit accounts table

In [2]:
reddit_account = RedditAccount(subreddit=subreddit, ddb=ddb)
daily_upload = DailyUpload(subreddit=subreddit)



In [3]:
reddit_account.fetch_and_update_account_details(REDDIT_ACCOUNTS_TABLE_NAME)

## Authorise with reddit

In [4]:
reddit_account.authenticate_with_api()

In [5]:
res = requests.post(REDDIT_AUTH_URL, auth=reddit_account.auth, data=reddit_account.data, headers= reddit_account.headers)
# reddit_account.data
res

<Response [200]>

In [6]:
reddit_account.fetch_access_token(REDDIT_AUTH_URL)

## Send request for posts (with params, limit 100 posts)

In [7]:

REDDIT_API_URL_TOP = REDDIT_API_URL_TOP.replace("placeholder_value", subreddit)
top_posts = reddit_account.fetch_posts_as_json(REDDIT_API_URL_TOP, params = {'limit': '100'})


In [8]:
len(top_posts['data']['children'])

100

In [9]:
daily_upload.parse_posts(top_posts)

In [11]:
from datetime import datetime
# daily_upload.serialize_to_item()


In [13]:
res = ddb.transact_write_items(
        TransactItems=[
            {
                "Put": {
                    "TableName": DAILY_UPLOADS_TABLE,
                    "Item": daily_upload.serialize_to_item(),
                }
            },
            {
                "Update": {
                    "TableName": DAILY_UPLOADS_TABLE,
                    "Key": {
                        "PK": {"S": daily_upload.date},
                        "SK": {"S": "todays_subreddits_count"},
                    },
                    "ConditionExpression": "attribute_exists(PK)",
                    "UpdateExpression": "SET #count = #count + :inc",
                    "ExpressionAttributeNames": {"#count": "count"},
                    "ExpressionAttributeValues": {":inc": {"N": "1"}},
                }
            },
        ]
    )

## Build table of urls

In [9]:
daily_upload.parse_posts(posts)

In [11]:
daily_upload.total_duration

1274

In [15]:
daily_upload.df_top

,title,upvote_ratio,ups,downs,score,url
0,Failing upwards,0.96,11881.0,0.0,11881.0,https://v.redd.it/npaaq8v7flg71
1,Falling in love!,0.95,7233.0,0.0,7233.0,https://v.redd.it/j4z3i5hwnjg71
2,What could go wrong?,0.92,3233.0,0.0,3233.0,https://v.redd.it/iqhld5kfxjg71
3,Damn,0.95,1647.0,0.0,1647.0,https://v.redd.it/5hwnpovhwog71
4,He will never get the vac,0.94,1850.0,0.0,1850.0,https://v.redd.it/koqooxdrbpg71
5,Guy annoys girlfriend with puns at IKEA,0.90,1006.0,0.0,1006.0,https://v.redd.it/4ryovpc4zog71
6,AAAAAAAAAAA,0.93,840.0,0.0,840.0,https://v.redd.it/oq956fvv5og71
7,Fighting and air monster.,0.87,523.0,0.0,523.0,https://v.redd.it/nwubgnzjapg71
8,Just Keanu in his overly priced apartment...,0.82,390.0,0.0,390.0,https://v.redd.it/48rka6l3jkg71
9,"Open the door, I forgot my keys!",0.95,372.0,0.0,372.0,https://v.redd.it/d9yqdwsibpg71


In [284]:
next_posts = reddit_account.fetch_posts_as_json(REDDIT_API_URL_TOP, params = {'limit': '100', 'after': daily_upload.latest_post['data']['name']})

In [285]:
post = next_posts['data']['children'][0] None,

In [287]:


hide_score = false, ups > 0 and num_comments > 0, over_18 false,  stickied false

if mod_note not None or removed_by not None or removal_reason not None, check if it has got comments, if

yes allow ?

def __removed_post_is_worthy(post):
    if post['mod_note'] not None or  post['removed_by'] not None or
     post['removal_reason']:
            if post['num_comments'] > 5 and post['score'] > 10:
                return True
            
    return False

def __is_eligible(post):
    if post['is_video'] and post['ups'] > 0 and post['num_comments'] > 0 and not post['over_18'] and not post['stickied']:
        return True
    
    return False



{'approved_at_utc': None,
 'subreddit': 'funny',
 'selftext': '',
 'author_fullname': 't2_3y9j6p39',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'catch 22(?)',
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/funny',
 'hidden': False,
 'pwls': 6,
 'link_flair_css_class': '',
 'downs': 0,
 'thumbnail_height': 139,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_p21yrk',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'upvote_ratio': 0.74,
 'author_flair_background_color': None,
 'ups': 46,
 'total_awards_received': 0,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': True,
 'is_meta': False,
 'category': None,
 'secure_media_embed': {},
 'link_flair_text': '!Rule 2 - Meme/memetic content - Removed',
 'can_mod_post': False,
 'score': 46,
 'approved_by': None,
 'is_created_from_ads_ui

In [288]:
if not None:
    print("h")

h


In [18]:
import time 
time.strftime(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(1628665616)))

'2021-08-11 12:36:56'

In [29]:
posts['data']['children'][0]['data']

False

In [31]:
from psaw import PushshiftAPI

In [32]:
api = PushshiftAPI()

In [157]:
from datetime import datetime, timedelta

In [158]:
today =  datetime.today()
yesterday = datetime.today() - timedelta(days=1)
day_before_yesterday = yesterday - timedelta(days=1)

In [159]:
today = datetime(today.year, today.month, today.day, 0,0,0).timestamp()
yesterday = datetime(yesterday.year, yesterday.month, yesterday.day,0,0,0).timestamp()
day_before_yesterday = datetime(day_before_yesterday.year, day_before_yesterday.month, day_before_yesterday.day,0,0,0).timestamp()

In [92]:
 a = list(api.search_submissions(after=day_before_yesterday, before=today, subreddit='funny', filter=['url', 'title'], limit = 10))

KeyboardInterrupt: 

In [201]:
int(day_before_yesterday)

1628447400

In [235]:
def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/submission/?subreddit=funny&num_comments=>0&after={int(day_before_yesterday)}&before={int(yesterday)}&is_video=true&sort_type=score&sort=score:asc&size=100&aggs=subreddit"
#     payload = {}
#     print(payload)
    request = requests.get(base_url)
    return request.json()

In [236]:
data_type="submission"     # give me comments, use "submission" to publish something
query="funny"          # Add your query
duration="1d"          # Select the timeframe. Epoch value or Integer + "s,m,h,d" (i.e. "second", "minute", "hour", "day")
size=1000               # maximum 1000 comments
sort_type="score"       # Sort by score (Accepted: "score", "num_comments", "created_utc")
sort="desc"             # sort descending
aggs="subreddit"        #"author", "link_id", "created_utc", "subreddit"

In [254]:
base_url = f"https://api.pushshift.io/reddit/search/submission/?subreddit=funny&num_comments=>0&over_18=false&after={int(day_before_yesterday)}&before={int(yesterday)}&is_video=true&sort_type=score&sort=score:desc&size=100&aggs=subreddit"

request = requests.get(base_url)
b = request.json()

In [263]:
b['data'][59]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'THCv1',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_1bfzjru7',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1628530642,
 'domain': '/r/funny/comments/p16ap5/we_all_have_at_least_one_in_every_town/',
 'full_link': 'https://www.reddit.com/r/funny/comments/p16ap5/we_all_have_at_least_one_in_every_town/',
 'gildings': {},
 'id': 'p16ap5',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': True,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': Fal

In [238]:
for post in a['data']:    
    pp.pprint(post)
    break

   
    
        


{ 'all_awardings': [],
  'allow_live_comments': False,
  'author': 'Tinzeii',
  'author_flair_css_class': None,
  'author_flair_richtext': [],
  'author_flair_text': None,
  'author_flair_type': 'text',
  'author_fullname': 't2_4fknz7x5',
  'author_is_blocked': False,
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'can_mod_post': False,
  'contest_mode': False,
  'created_utc': 1628460924,
  'domain': '/r/funny/comments/p0o96c/enjoy_ittt/',
  'full_link': 'https://www.reddit.com/r/funny/comments/p0o96c/enjoy_ittt/',
  'gildings': {},
  'id': 'p0o96c',
  'is_created_from_ads_ui': False,
  'is_crosspostable': True,
  'is_meta': False,
  'is_original_content': False,
  'is_reddit_media_domain': False,
  'is_robot_indexable': True,
  'is_self': False,
  'is_video': True,
  'link_flair_background_color': '',
  'link_flair_richtext': [],
  'link_flair_text_color': 'dark',
  'link_flair_type': 'text',
  'locked': False,
  'media_only': False,
  'no_follow': Tr

In [239]:
post

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Tinzeii',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_4fknz7x5',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1628460924,
 'domain': '/r/funny/comments/p0o96c/enjoy_ittt/',
 'full_link': 'https://www.reddit.com/r/funny/comments/p0o96c/enjoy_ittt/',
 'gildings': {},
 'id': 'p0o96c',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': True,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 7,
 'num_cros